In [1]:
''' read & compute SD alongtrack timeseries of GFS & SD's wind power and acc energy to NI motions '''
import numpy as np
import datetime
import os
import sys
from scipy import stats

In [7]:
''' SD data info - partial track data '''
year = '2023'
platfs_num = ['1036','1041','1069'] #['1031','1036','1040','1041','1042','1045','1057','1064','1065','1068','1069','1083']
path_SD = os.path.expanduser('~/Documents/projects/sd-ni-wp/HU_LEE/')
filenames_SD_all = np.sort( os.listdir(path_SD) )
filenames_SD = np.array([])
for file in filenames_SD_all:
    if ('timeseries' in file) & ('.txt' in file) & (year in file):
        filenames_SD = np.append(filenames_SD, file)
print(len(filenames_SD), filenames_SD)

3 ['timeseries_wnd_cur_bp(0.80f-1.20f)_wp_energy_2023-SD1036_20230907-0919_600.0-sec-grid.txt'
 'timeseries_wnd_cur_bp(0.80f-1.20f)_wp_energy_2023-SD1041_20230906-0918_600.0-sec-grid.txt'
 'timeseries_wnd_cur_bp(0.80f-1.20f)_wp_energy_2023-SD1069_20230903-0915_600.0-sec-grid.txt']


In [8]:
''' along track data info with GFS flux data '''
path_GFS = os.path.expanduser('~/Documents/projects/sd-ni-wp/SD_windstress/compare_SD_model/data_timeseries_match_SD_GFS/')
filenames_GFS_all = np.sort( os.listdir(path_GFS) )
filenames_GFS = np.array([])
for file in filenames_GFS_all:
    if ('u10' in file) & ('.txt' in file) & (year in file) & (file[-12:-8] in platfs_num):
        filenames_GFS = np.append(filenames_GFS, file)
print(len(filenames_GFS), filenames_GFS)

3 ['timeseries_wnd_u10m_match_2023-SD1036_GFS.txt'
 'timeseries_wnd_u10m_match_2023-SD1041_GFS.txt'
 'timeseries_wnd_u10m_match_2023-SD1069_GFS.txt']


In [ ]:
''' compute & compare/ print results from SD & model '''
for i, platf in enumerate( platfs_num ):
    dataGFS = np.genfromtxt( path_GFS+filenames_GFS[i], skip_header=True, delimiter=' ')
    dataSD = np.genfromtxt( path_SD+filenames_SD[i], skip_header=True, delimiter=' ')
    print('Look at 2 files:',filenames_GFS[i], filenames_SD[i])
    dtimeGFS = np.array([datetime.datetime.fromtimestamp(item) for item in dataGFS[:,0]])
    print('GFS time:',dtimeGFS[0:2],'...', dtimeGFS[-1])
    dtimeSD = np.array([datetime.datetime.fromtimestamp(item) for item in dataSD[:,0]])
    print('SD time:',dtimeSD[0:2],'...', dtimeSD[-1])
    ''' find match data points: GFS is hourly data '''

    ''' compute 2 sets of accumulated energy from SD: (1) on hourly grid (2) on 10-minute grid '''

    ''' plot & print GFS (from UFLX, VFLX) & SD's (2 temporal grid) wind power & accumulated energy results '''
    break

Look at 2 files: timeseries_wnd_u10m_match_2023-SD1036_GFS.txt timeseries_wnd_cur_bp(0.80f-1.20f)_wp_energy_2023-SD1036_20230907-0919_600.0-sec-grid.txt
GFS time: [datetime.datetime(2023, 8, 1, 1, 0) datetime.datetime(2023, 8, 1, 2, 0)] ... 2023-10-26 18:00:00
SD time: [datetime.datetime(2023, 9, 7, 0, 0) datetime.datetime(2023, 9, 7, 0, 10)] ... 2023-09-18 23:50:00
